<a href="https://colab.research.google.com/github/RicAntonio/Car_Fuel_and_Emissions_2000-2013_Project/blob/main/Car_Fuel_and_Emissions_2000_2013_Clasificaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

CAR_FUEL_AND_EMISSIONS_2000-2013

In [5]:
# feature_pipeline_colab.py

import pandas as pd
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive')

def load_data(csv_path):
    return pd.read_csv(csv_path)

def process_data(data):
    # Seleccionamos características útiles y eliminamos filas nulas
    data = data[["year", "manufacturer", "model", "description",
                 "euro_standard", "tax_band", "transmission",
                 "transmission_type", "engine_capacity", "fuel_type",
                 "urban_metric", "extra_urban_metric", "combined_metric",
                 "urban_imperial", "extra_urban_imperial",
                 "combined_imperial", "noise_level", "co2" ,
                 "thc_emissions"]].dropna()

    # Convertimos las variables categóricas a numéricas
    data = pd.get_dummies(data, columns=["fuel_type", "transmission_type",
                                         "manufacturer"])

    return data

def save_features(features, feature_store_path):
    features.to_csv(feature_store_path, index=False)

if __name__ == "__main__":
    # Cargar datos desde Google Drive
    data = load_data("/content/drive/MyDrive/Car_Fuel_and_Emissions_2000-2013.csv")
    features = process_data(data)

    # Guardar en Google Drive
    save_features(features, "/content/drive/MyDrive/features_car_fuel_emissions.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-5-cbd61acf5fc9>:10: DtypeWarning: Columns (6,30) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_path)


In [9]:
print(features.columns)


Index(['year', 'model', 'description', 'euro_standard', 'tax_band',
       'transmission', 'engine_capacity', 'urban_metric', 'extra_urban_metric',
       'combined_metric', 'urban_imperial', 'extra_urban_imperial',
       'combined_imperial', 'noise_level', 'co2', 'thc_emissions',
       'fuel_type_CNG', 'fuel_type_Petrol',
       'fuel_type_Petrol / E85 (Flex Fuel)', 'fuel_type_Petrol Electric',
       'fuel_type_Petrol Hybrid', 'transmission_type_Automatic',
       'transmission_type_Manual', 'manufacturer_Abarth',
       'manufacturer_Alfa Romeo', 'manufacturer_Aston Martin Lagonda',
       'manufacturer_Audi', 'manufacturer_BMW', 'manufacturer_Bentley Motors',
       'manufacturer_Chevrolet', 'manufacturer_Chrysler Jeep',
       'manufacturer_Citroen', 'manufacturer_Corvette', 'manufacturer_Dacia',
       'manufacturer_Ferrari', 'manufacturer_Fiat', 'manufacturer_Ford',
       'manufacturer_Honda', 'manufacturer_Hyundai', 'manufacturer_Infiniti',
       'manufacturer_Jaguar Cars',

In [12]:
# training_pipeline_colab.py

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import pickle
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

def load_features(feature_store_path):
    return pd.read_csv(feature_store_path)

def preprocess_features(features):
    # Convertir variables categóricas en variables dummy (One-Hot Encoding)
    features_encoded = pd.get_dummies(features)
    return features_encoded

def train_model(features, target_column):
    X = features.drop(columns=[target_column])
    y = features[target_column]

    # Dividimos los datos en entrenamiento y validación
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenamos un clasificador
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)

    return model

def save_model(model, model_registry_path):
    with open(model_registry_path, 'wb') as f:
        pickle.dump(model, f)

if __name__ == "__main__":
    # Cargar características desde Google Drive
    features = load_features("/content/drive/MyDrive/features_car_fuel_emissions.csv")

    # Imprimir las columnas para verificar los nombres
    print("Columnas antes del procesamiento:", features.columns)

    # Preprocesar características (convertir categóricas a numéricas)
    features_encoded = preprocess_features(features)

    # Imprimir las columnas después del procesamiento
    print("Columnas después del procesamiento:", features_encoded.columns)

    # Entrenar el modelo (usamos "Fuel Type_Petrol" como columna objetivo)
    model = train_model(features_encoded, "fuel_type_Petrol")  # Cambia según tu columna de destino

    # Guardar el modelo en Google Drive
    save_model(model, "/content/drive/MyDrive/model_car_fuel_type.pkl")


Mounted at /content/drive
Columnas antes del procesamiento: Index(['year', 'model', 'description', 'euro_standard', 'tax_band',
       'transmission', 'engine_capacity', 'urban_metric', 'extra_urban_metric',
       'combined_metric', 'urban_imperial', 'extra_urban_imperial',
       'combined_imperial', 'noise_level', 'co2', 'thc_emissions',
       'fuel_type_CNG', 'fuel_type_Petrol',
       'fuel_type_Petrol / E85 (Flex Fuel)', 'fuel_type_Petrol Electric',
       'fuel_type_Petrol Hybrid', 'transmission_type_Automatic',
       'transmission_type_Manual', 'manufacturer_Abarth',
       'manufacturer_Alfa Romeo', 'manufacturer_Aston Martin Lagonda',
       'manufacturer_Audi', 'manufacturer_BMW', 'manufacturer_Bentley Motors',
       'manufacturer_Chevrolet', 'manufacturer_Chrysler Jeep',
       'manufacturer_Citroen', 'manufacturer_Corvette', 'manufacturer_Dacia',
       'manufacturer_Ferrari', 'manufacturer_Fiat', 'manufacturer_Ford',
       'manufacturer_Honda', 'manufacturer_Hyundai',

In [14]:
# batch_inference_pipeline_colab.py

import pandas as pd
import pickle
from google.colab import drive

# Montar Google Drive
drive.mount('/content/drive', force_remount=True)

def load_model(model_registry_path):
    with open(model_registry_path, 'rb') as f:
        model = pickle.load(f)
    return model

def load_latest_features(feature_store_path):
    return pd.read_csv(feature_store_path)

def save_predictions(predictions, output_path):
    predictions.to_csv(output_path, index=False)

def align_features(features, expected_columns):
    """
    Alinea las características actuales con las columnas esperadas por el modelo.
    Si faltan columnas, rellena con ceros. Si hay columnas adicionales, las elimina.
    """
    # Rellenar las columnas faltantes con ceros
    for col in expected_columns:
        if col not in features.columns:
            features[col] = 0

    # Eliminar columnas adicionales que no están en las columnas esperadas
    features = features[expected_columns]

    return features

if __name__ == "__main__":
    # Cargar el modelo desde Google Drive
    model = load_model("/content/drive/MyDrive/model_car_fuel_type.pkl")

    # Cargar las últimas características desde Google Drive
    features = load_latest_features("/content/drive/MyDrive/features_car_fuel_emissions.csv")

    # Obtener los nombres de las columnas que el modelo espera
    expected_columns = model.feature_names_in_

    # Alinear las características con las columnas que espera el modelo
    X = align_features(features, expected_columns)

    # Generar las predicciones
    predictions = model.predict(X)

    # Guardar las predicciones en Google Drive
    output = pd.DataFrame(predictions, columns=["Predicted Fuel Type"])
    save_predictions(output, "/content/drive/MyDrive/predictions_car_fuel_type.csv")


Mounted at /content/drive


<ipython-input-14-26b3d2d0461b>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col] = 0
<ipython-input-14-26b3d2d0461b>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col] = 0
<ipython-input-14-26b3d2d0461b>:29: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features[col] = 0
<